# Guia de Uso

1. Base de dados Grande e Diferente = Treinar todo o modelo
2. Base de dados Grande e Parecida = Fine Tuning
3. Base de dados Pequena e Diferente = Fine Tuning
4. Base de dados Pequena e Parecida = Transferência de Aprendizagem

# Transferência de Aprendizagem com Redes Neurais

## Importando as bibliotecas e base de dados

In [2]:

import os
import scipy
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Fazendo o download da base de dados de gatos e cachorros

In [1]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O ./cats_and_dogs_filtered.zip

--2022-09-26 15:02:02--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolvendo storage.googleapis.com (storage.googleapis.com)... 172.217.162.208, 172.217.173.80, 172.217.173.112, ...
Conectando-se a storage.googleapis.com (storage.googleapis.com)|172.217.162.208|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 68606236 (65M) [application/zip]
Salvando em: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65,43M  5,31MB/s    em 11s     

2022-09-26 15:02:15 (5,83 MB/s) - ‘./cats_and_dogs_filtered.zip’ salvo [68606236/68606236]



## Pré-processamento

### Descompactando a base de dados de gatos e cachorros

In [33]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [34]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [35]:
zip_object.extractall("./")

In [36]:
zip_object.close()

### Configurando os caminhos (paths)

In [16]:
dataset_path_new = "./cats_and_dogs_filtered"

In [17]:
train_dir = os.path.join(dataset_path_new, 'train')

In [18]:
validation_dir = os.path.join(dataset_path_new, 'validation')

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [30]:
img_shape = (128, 128, 3)

In [31]:
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape, include_top=False, weights='imagenet')

In [32]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 16, 16, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 16, 16, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 16, 16, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 16, 16, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 8, 8, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 8, 8, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 8, 8, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 8, 8, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 8, 8, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 8, 8, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 8, 8, 576)    0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 4, 4, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 4, 4, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 4, 4, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 4, 4, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

### Congelando o modelo base

In [33]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [34]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [35]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [36]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_1')>

In [37]:
prediction_layer = tf.keras.layers.Dense(units=1, activation="sigmoid")(global_average_layer)

### Definindo o modelo

In [38]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [39]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']         

 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 16, 16, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_depthwise_relu (ReLU)  (None, 16, 16, 144)  0           ['block_3_depthwise_BN[0][0]']   
                                                                                                  
 block_3_project (Conv2D)       (None, 16, 16, 32)   4608        ['block_3_depthwise_relu[0][0]'] 
                                                                                                  
 block_3_project_BN (BatchNorma  (None, 16, 16, 32)  128         ['block_3_project[0][0]']        
 lization)

 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 8, 8, 384)    0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 8, 8, 384)   3456        ['block_7_expand_relu[0][0]']    
 nv2D)                                                                                            
                                                                                                  
 block_7_depthwise_BN (BatchNor  (None, 8, 8, 384)   1536        ['block_7_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_7_d

 block_10_project_BN (BatchNorm  (None, 8, 8, 96)    384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 8, 8, 576)    55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_expand_BN (BatchNorma  (None, 8, 8, 576)   2304        ['block_11_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_11_expand_relu (ReLU)    (None, 8, 8, 576)    0           ['block_11_expand_BN[0][0]']     
                                                                                                  
 block_11_

                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 4, 4, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 4, 4, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_project_BN (BatchNorm  (None, 4, 4, 160)   640         ['block_14_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_14_add (Add)             (None, 4, 4, 160)    0           ['block_13_project_BN[0][0]',    
                                                                  'block_14_project_BN[0][0]']    
          

### Compilando o modelo

In [40]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss="binary_crossentropy", metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Rendimensionando as imagens

    - Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.
    
Por exemplo: MobileNet (que estamos usando) suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224)

In [41]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_valid = ImageDataGenerator(rescale=1/255.0)

In [42]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [43]:
valid_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(128, 128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Treinando o modelo

In [44]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 7s 358ms/step - loss: 0.6622 - accuracy: 0.6130 - val_loss: 0.6070 - val_accuracy: 0.6710
Epoch 2/5
16/16 [==============================] - 5s 330ms/step - loss: 0.5600 - accuracy: 0.7140 - val_loss: 0.5265 - val_accuracy: 0.7510
Epoch 3/5
16/16 [==============================] - 5s 328ms/step - loss: 0.4877 - accuracy: 0.7845 - val_loss: 0.4614 - val_accuracy: 0.8100
Epoch 4/5
16/16 [==============================] - 5s 328ms/step - loss: 0.4274 - accuracy: 0.8270 - val_loss: 0.4062 - val_accuracy: 0.8490
Epoch 5/5
16/16 [==============================] - 5s 328ms/step - loss: 0.3779 - accuracy: 0.8555 - val_loss: 0.3607 - val_accuracy: 0.8760


### Avaliação do modelo de transferência de aprendizagem

In [45]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 2s 210ms/step - loss: 0.3607 - accuracy: 0.8760


In [46]:
valid_accuracy

0.8759999871253967

In [47]:
valid_loss

0.36074715852737427

# Fine Tuning

Duas questões principais:

- NÃO USE Fine Tuning em toda a rede neural, pois somente em algumas camadas já é suficiente. A ideia é adotar parte específica da rede neural para nosso problema específico.


- Inicie o Fine Tuning DEPOIS que você finalizou a transferência de aprendizagem. Se você tentar o Fine Tuning imediatamente, os gradientes serão muito diferentes entre o cabeçalho personalizado e algumas camadas descongeladas do modelo base

### Descongelando algumas camadas do topo do modelo base

In [50]:
base_model.trainable = True
len(base_model.layers)

154

In [51]:
fine_tuning_at = 100

In [52]:
for layer in base_model.layers[:fine_tuning_at]:
    layer.trainable = False

### Compilando o modelo para o fine tuning

In [54]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Fine tuning

In [56]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 10s 472ms/step - loss: 0.1905 - accuracy: 0.9265 - val_loss: 0.0948 - val_accuracy: 0.9650
Epoch 2/5
16/16 [==============================] - 7s 443ms/step - loss: 0.0274 - accuracy: 0.9970 - val_loss: 0.0864 - val_accuracy: 0.9710
Epoch 3/5
16/16 [==============================] - 7s 430ms/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 0.1115 - val_accuracy: 0.9660
Epoch 4/5
16/16 [==============================] - 7s 448ms/step - loss: 0.0050 - accuracy: 0.9995 - val_loss: 0.1256 - val_accuracy: 0.9660
Epoch 5/5
16/16 [==============================] - 7s 432ms/step - loss: 0.0020 - accuracy: 1.0000 - val_loss: 0.1170 - val_accuracy: 0.9670


### Avaliação do modelo com fine tuning

In [57]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 2s 220ms/step - loss: 0.1170 - accuracy: 0.9670


In [58]:
valid_accuracy

0.9670000076293945

In [59]:
valid_loss

0.11699822545051575